In [1]:
from google.cloud import translate
import time
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
client = translate.TranslationServiceClient()



In [2]:
def praphrase_sentences(text, depth=10,
                        project_id="complex-app-300ee"):
    parent = client.location_path(project_id, "global")
    x = client.get_supported_languages(parent)
    target_laguages = [item.language_code for item in x.languages[:depth]]
    translated_text = []
    for language in target_laguages:
        response = client.translate_text(
            parent=parent,
            contents=[text],
            mime_type='text/plain',  # mime types: text/plain, text/html
            source_language_code='en-IN',
            target_language_code=language)
        for translation in response.translations:
            translated_text.append(translation.translated_text)
    result = []
    for lg, sentence in zip(target_laguages, translated_text):
        response = client.translate_text(
            parent=parent,
            contents=[sentence],
            mime_type='text/plain',  # mime types: text/plain, text/html
            source_language_code=str(lg),
            target_language_code="en")
        for translation in response.translations:
            result.append(translation.translated_text)
    return result

In [3]:
def make_parahrases(sentences):
    client = translate.TranslationServiceClient()
    if len(sentences) != 0:
        s=time.time()
        threads=len(sentences)+1 if len(sentences)< multiprocessing.cpu_count()-1 else multiprocessing.cpu_count()-1
        with ThreadPoolExecutor(max_workers=threads) as executore:
            future=executore.map(praphrase_sentences,sentences)
        return [list(set(item)) for item in future]
    

In [4]:
sentences=["hello how are you"]

In [5]:
s= time.time()
make_parahrases(sentences)
print("Time for paraphrasing 50 sentences={}".format(time.time()-s))

Time for paraphrasing 50 sentences=6.681522607803345


# Best method

In [6]:
from google.cloud import translate
import time
from concurrent.futures import ThreadPoolExecutor
import multiprocessing

In [7]:
client = translate.TranslationServiceClient()
parent = client.location_path("complex-app-300ee", "global")
target_laguages = [item.language_code for item in client.get_supported_languages(parent).languages[:10]]

def praphrase_sentences(text):
    global parent,target_laguages
    translated_text = []
    
    def get_the_other(language):
        response = client.translate_text(
        parent=parent,
        contents=[text],
        mime_type='text/plain',  # mime types: text/plain, text/html
        source_language_code='en-IN',
        target_language_code=language)
        translated_text=[]
        for translation in response.translations:
            translated_text.append(translation.translated_text)
        return translated_text
    
    with ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()-1) as ex:
        result=ex.map(get_the_other,target_laguages)
    translated_text=[]
    for item in result:
        translated_text.extend(item)
    result = []
    for lg, sentence in zip(target_laguages, translated_text):
        response = client.translate_text(
            parent=parent,
            contents=[sentence],
            mime_type='text/plain',  # mime types: text/plain, text/html
            source_language_code=str(lg),
            target_language_code="en")
        for translation in response.translations:
            result.append(translation.translated_text)
    return result
        
        
    


In [8]:
def get_list_paraphrase(sentences):
    threads = len(sentences) if len(sentences) < multiprocessing.cpu_count() else multiprocessing.cpu_count()-1
    with ThreadPoolExecutor(max_workers=threads) as exe:
        future=exe.map(praphrase_sentences,sentences)
    return [list(set(item)) for item in future]

In [9]:
sentences=["hello how are you"]

In [10]:
s=time.time()
x=get_list_paraphrase(sentences*100)
print("Time for paraphrasing 2 sentences={}".format(time.time()-s))

Time for paraphrasing 2 sentences=16.407154321670532


In [11]:
x

[['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, you',
  'Hello how are you',
  'Hello how is it going'],
 ['hello there',
  'Greetings, y